In [15]:
import json
import wandb
import transformers
import torch
import gc
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    Trainer,
    TrainingArguments,
    DataCollatorForLanguageModeling
)
from datasets import Dataset
from safetensors.torch import load_model, save_model

model_path = "checkpoints/2b-functions-full/checkpoint-4000/"

tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    use_safetensors=True,
)
model.eval()

Loading checkpoint shards: 100%|██████████| 2/2 [00:10<00:00,  5.38s/it]


Gemma2ForCausalLM(
  (model): Gemma2Model(
    (embed_tokens): Embedding(256000, 2304, padding_idx=0)
    (layers): ModuleList(
      (0-25): 26 x Gemma2DecoderLayer(
        (self_attn): Gemma2Attention(
          (q_proj): Linear(in_features=2304, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2304, out_features=1024, bias=False)
          (v_proj): Linear(in_features=2304, out_features=1024, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2304, bias=False)
        )
        (mlp): Gemma2MLP(
          (gate_proj): Linear(in_features=2304, out_features=9216, bias=False)
          (up_proj): Linear(in_features=2304, out_features=9216, bias=False)
          (down_proj): Linear(in_features=9216, out_features=2304, bias=False)
          (act_fn): PytorchGELUTanh()
        )
        (input_layernorm): Gemma2RMSNorm((2304,), eps=1e-06)
        (post_attention_layernorm): Gemma2RMSNorm((2304,), eps=1e-06)
        (pre_feedforward_layernorm): Gemm

In [125]:
prompt = "What is the value of ydmsml(100)?"
messages = [{"role": "user", "content": prompt}]
inputs = tokenizer.apply_chat_template(
    messages,
    return_tensors="pt",
    padding=True,
    tokenize=True,
    add_generation_prompt=True,
).to(model.device)

outputs = model.generate(
    inputs,
    do_sample=False,
    max_length=256,
)

decoded_output = tokenizer.decode(outputs[0])
print(decoded_output)

<bos><start_of_turn>user
What is the value of ydmsml(100)?<end_of_turn>
<start_of_turn>model
89<end_of_turn>


In [ ]:
model.model.embed_tokens(inputs)

Gemma2ForCausalLM(
  (model): Gemma2Model(
    (embed_tokens): Embedding(256000, 2304, padding_idx=0)
    (layers): ModuleList(
      (0-25): 26 x Gemma2DecoderLayer(
        (self_attn): Gemma2Attention(
          (q_proj): Linear(in_features=2304, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2304, out_features=1024, bias=False)
          (v_proj): Linear(in_features=2304, out_features=1024, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2304, bias=False)
        )
        (mlp): Gemma2MLP(
          (gate_proj): Linear(in_features=2304, out_features=9216, bias=False)
          (up_proj): Linear(in_features=2304, out_features=9216, bias=False)
          (down_proj): Linear(in_features=9216, out_features=2304, bias=False)
          (act_fn): PytorchGELUTanh()
        )
        (input_layernorm): Gemma2RMSNorm((2304,), eps=1e-06)
        (post_attention_layernorm): Gemma2RMSNorm((2304,), eps=1e-06)
        (pre_feedforward_layernorm): Gemm

In [20]:
for name, param in model.named_parameters():
    print(name)

model.embed_tokens.weight
model.layers.0.self_attn.q_proj.weight
model.layers.0.self_attn.k_proj.weight
model.layers.0.self_attn.v_proj.weight
model.layers.0.self_attn.o_proj.weight
model.layers.0.mlp.gate_proj.weight
model.layers.0.mlp.up_proj.weight
model.layers.0.mlp.down_proj.weight
model.layers.0.input_layernorm.weight
model.layers.0.post_attention_layernorm.weight
model.layers.0.pre_feedforward_layernorm.weight
model.layers.0.post_feedforward_layernorm.weight
model.layers.1.self_attn.q_proj.weight
model.layers.1.self_attn.k_proj.weight
model.layers.1.self_attn.v_proj.weight
model.layers.1.self_attn.o_proj.weight
model.layers.1.mlp.gate_proj.weight
model.layers.1.mlp.up_proj.weight
model.layers.1.mlp.down_proj.weight
model.layers.1.input_layernorm.weight
model.layers.1.post_attention_layernorm.weight
model.layers.1.pre_feedforward_layernorm.weight
model.layers.1.post_feedforward_layernorm.weight
model.layers.2.self_attn.q_proj.weight
model.layers.2.self_attn.k_proj.weight
model.la

In [35]:
model.model.layers[0].input_layernorm.weight

Parameter containing:
tensor([0.1167, 0.1348, 0.1924,  ..., 0.6367, 0.0403, 0.2432], device='cuda:0',
       dtype=torch.bfloat16, requires_grad=True)

In [129]:
from transformer_lens import HookedTransformer

gemma2_original = HookedTransformer.from_pretrained('google/gemma-2-2b-it', center_unembed=False)

for name, param in gemma2_original.named_parameters():
    print(name)

Loading checkpoint shards: 100%|██████████| 2/2 [00:16<00:00,  8.32s/it]


Loaded pretrained model google/gemma-2-2b-it into HookedTransformer
embed.W_E
blocks.0.ln1_post.w
blocks.0.ln2_post.w
blocks.0.attn.W_Q
blocks.0.attn.W_O
blocks.0.attn.b_Q
blocks.0.attn.b_O
blocks.0.attn._W_K
blocks.0.attn._W_V
blocks.0.attn._b_K
blocks.0.attn._b_V
blocks.0.mlp.W_in
blocks.0.mlp.W_out
blocks.0.mlp.W_gate
blocks.0.mlp.b_in
blocks.0.mlp.b_out
blocks.1.ln1_post.w
blocks.1.ln2_post.w
blocks.1.attn.W_Q
blocks.1.attn.W_O
blocks.1.attn.b_Q
blocks.1.attn.b_O
blocks.1.attn._W_K
blocks.1.attn._W_V
blocks.1.attn._b_K
blocks.1.attn._b_V
blocks.1.mlp.W_in
blocks.1.mlp.W_out
blocks.1.mlp.W_gate
blocks.1.mlp.b_in
blocks.1.mlp.b_out
blocks.2.ln1_post.w
blocks.2.ln2_post.w
blocks.2.attn.W_Q
blocks.2.attn.W_O
blocks.2.attn.b_Q
blocks.2.attn.b_O
blocks.2.attn._W_K
blocks.2.attn._W_V
blocks.2.attn._b_K
blocks.2.attn._b_V
blocks.2.mlp.W_in
blocks.2.mlp.W_out
blocks.2.mlp.W_gate
blocks.2.mlp.b_in
blocks.2.mlp.b_out
blocks.3.ln1_post.w
blocks.3.ln2_post.w
blocks.3.attn.W_Q
blocks.3.attn.W_O


In [132]:
gemma2_original.unembed.W_U - gemma2_original.embed.W_E.T

tensor([[-1.5705,  0.8943,  0.0073,  ..., -1.6577, -0.5208, -1.6141],
        [ 1.0248, -2.3331,  0.2644,  ...,  1.0575,  1.8316,  1.1720],
        [-3.6465,  1.3538, -0.9935,  ..., -2.1726, -1.1300, -3.5155],
        ...,
        [-0.8942, -0.1176, -0.6435,  ..., -0.3166,  0.1034, -0.9097],
        [-2.3450,  1.0694,  0.3271,  ..., -0.2836, -2.0193, -2.3342],
        [ 1.5667,  0.7724,  0.5304,  ..., -0.5957, -0.9737,  1.6319]],
       device='cuda:0', grad_fn=<SubBackward0>)

In [47]:
from torch import nn

def convert_weight_name(source_name):
    # Remove trailing '.weight' if present
    if source_name.endswith(".weight"):
        source_name = source_name[:-7]
    
    # Split the name into components
    parts = source_name.split(".")
    
    # Handle model.embed_tokens -> embed.W_E
    if len(parts) == 2 and parts[0] == "model" and parts[1] == "embed_tokens":
        return "embed.W_E"
    
    # Handle layer-specific weights
    if len(parts) >= 4 and parts[0] == "model" and parts[1] == "layers":
        layer_num = parts[2]
        component = parts[3]
        
        # Handle attention weights
        if component == "self_attn":
            proj_type = parts[4]
            if proj_type == "q_proj":
                return f"blocks.{layer_num}.attn.W_Q"
            elif proj_type == "k_proj":
                return f"blocks.{layer_num}.attn._W_K"
            elif proj_type == "v_proj":
                return f"blocks.{layer_num}.attn._W_V"
            elif proj_type == "o_proj":
                return f"blocks.{layer_num}.attn.W_O"
        
        # Handle MLP weights
        elif component == "mlp":
            proj_type = parts[4]
            if proj_type == "gate_proj":
                return f"blocks.{layer_num}.mlp.W_gate"
            elif proj_type == "up_proj":
                return f"blocks.{layer_num}.mlp.W_in"
            elif proj_type == "down_proj":
                return f"blocks.{layer_num}.mlp.W_out"
        
        # Handle layer norms
        elif component == "post_attention_layernorm":
            return f"blocks.{layer_num}.ln1_post.w"
        elif component == "post_feedforward_layernorm":
            return f"blocks.{layer_num}.ln2_post.w"
    
    # If no conversion rule matches, return the original name
    return None


for name, param in model.named_parameters():
    converted_name = convert_weight_name(name)
    if converted_name is None:
        print(f"No conversion rule for {name}")
        continue
    
    if isinstance(param, nn.Parameter):
        param = param.data
    
    shape = gemma2_original.state_dict()[converted_name].shape
    param = param.view(shape)

    print(f"Loading {converted_name}")
    gemma2_original.state_dict()[converted_name].copy_(param)

Loading embed.W_E
Loading blocks.0.attn.W_Q
Loading blocks.0.attn._W_K
Loading blocks.0.attn._W_V
Loading blocks.0.attn.W_O
Loading blocks.0.mlp.W_gate
Loading blocks.0.mlp.W_in
Loading blocks.0.mlp.W_out
No conversion rule for model.layers.0.input_layernorm.weight
Loading blocks.0.ln1_post.w
No conversion rule for model.layers.0.pre_feedforward_layernorm.weight
Loading blocks.0.ln2_post.w
Loading blocks.1.attn.W_Q
Loading blocks.1.attn._W_K
Loading blocks.1.attn._W_V
Loading blocks.1.attn.W_O
Loading blocks.1.mlp.W_gate
Loading blocks.1.mlp.W_in
Loading blocks.1.mlp.W_out
No conversion rule for model.layers.1.input_layernorm.weight
Loading blocks.1.ln1_post.w
No conversion rule for model.layers.1.pre_feedforward_layernorm.weight
Loading blocks.1.ln2_post.w
Loading blocks.2.attn.W_Q
Loading blocks.2.attn._W_K
Loading blocks.2.attn._W_V
Loading blocks.2.attn.W_O
Loading blocks.2.mlp.W_gate
Loading blocks.2.mlp.W_in
Loading blocks.2.mlp.W_out
No conversion rule for model.layers.2.input_l

In [107]:
gemma2_original.state_dict()["unembed.W_U"].copy_(model.model.embed_tokens.weight.T)

tensor([[ 3.5156e-02, -2.0020e-02, -2.1219e-05,  ...,  3.7109e-02,
          1.1658e-02,  3.6133e-02],
        [-2.2949e-02,  5.2246e-02, -5.9204e-03,  ..., -2.3682e-02,
         -4.1016e-02, -2.6245e-02],
        [ 8.1543e-02, -3.0273e-02,  2.2217e-02,  ...,  4.8584e-02,
          2.5269e-02,  7.8613e-02],
        ...,
        [ 2.1118e-02,  2.7771e-03,  1.5198e-02,  ...,  7.4768e-03,
         -2.4414e-03,  2.1484e-02],
        [ 5.2734e-02, -2.4048e-02, -7.3547e-03,  ...,  6.3782e-03,
          4.5410e-02,  5.2490e-02],
        [-3.5156e-02, -1.7334e-02, -1.1902e-02,  ...,  1.3367e-02,
          2.1851e-02, -3.6621e-02]], device='cuda:0', grad_fn=<CopyBackwards>)

In [113]:
model.model.layers[0].self_attn.q_proj.weight

Parameter containing:
tensor([[-0.0070,  0.0149, -0.0193,  ...,  0.0148, -0.0278,  0.0087],
        [-0.0022,  0.0156, -0.0254,  ...,  0.0205, -0.0100,  0.0049],
        [ 0.0016,  0.0034,  0.0062,  ..., -0.0003,  0.0136, -0.0012],
        ...,
        [ 0.0030,  0.0276,  0.0087,  ...,  0.0175, -0.0114, -0.0014],
        [ 0.0042, -0.0025,  0.0055,  ...,  0.0070,  0.0003,  0.0060],
        [-0.0100, -0.0110,  0.0214,  ..., -0.0153,  0.0339, -0.0184]],
       device='cuda:0', dtype=torch.bfloat16, requires_grad=True)

In [ ]:
gemma2_original.parame

Parameter containing:
tensor([[[-7.0190e-03,  1.4893e-02, -1.9287e-02,  ..., -5.1880e-03,
          -1.7822e-02,  1.3351e-03],
         [ 1.6968e-02,  1.7578e-02, -2.1606e-02,  ...,  7.1411e-03,
          -3.4332e-03, -7.6294e-03],
         [-7.3853e-03,  6.3171e-03,  1.0803e-02,  ...,  1.3245e-02,
          -1.7700e-02, -5.5542e-03],
         ...,
         [ 6.4087e-03,  6.7444e-03,  1.1108e-02,  ...,  5.2795e-03,
           3.7537e-03,  1.5442e-02],
         [ 1.3123e-03, -1.2756e-02,  6.6757e-04,  ..., -8.9722e-03,
           8.1177e-03,  2.6245e-03],
         [-9.2163e-03,  1.5869e-03, -2.2278e-03,  ..., -8.0872e-04,
          -1.1658e-02,  5.1575e-03]],

        [[-4.9744e-03, -3.5858e-03, -1.2054e-03,  ...,  1.9379e-03,
           4.4861e-03,  2.0905e-03],
         [ 3.0823e-03,  5.7068e-03,  5.2643e-04,  ...,  1.9150e-03,
          -5.5542e-03,  2.8381e-03],
         [ 8.4686e-04, -6.8054e-03,  5.1270e-03,  ...,  6.1035e-03,
          -1.9302e-03, -1.7776e-03],
         ...,
   

In [108]:
prompt = "What is the value of ydmsml(17)?"
messages = [{"role": "user", "content": prompt}]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True,
)

In [109]:
logits, cache = gemma2_original.run_with_cache(inputs, remove_batch_dim=True)

tokens = gemma2_original.to_str_tokens(inputs)

In [66]:
import circuitsvis as cv

for k in range(3):
    print(f"Layer {k} attention pattern:")
    attention_pattern = cache["pattern", k]
    display(
        cv.attention.attention_patterns(
            tokens=tokens,
            attention=attention_pattern,
        )
    )

Layer 0 attention pattern:


Layer 1 attention pattern:


Layer 2 attention pattern:


In [80]:
m_prompt = "What is the value of ishfpw(100)?"
m_inputs = tokenizer.apply_chat_template(
    [{"role": "user", "content": m_prompt}],
    tokenize=False,
    add_generation_prompt=True,
)

m_logits, m_cache = gemma2_original.run_with_cache(m_inputs, remove_batch_dim=True)

m_tokens = gemma2_original.to_str_tokens(m_inputs)

In [84]:
cache['blocks.0.mlp.hook_post'].shape

torch.Size([22, 9216])

In [110]:
from jaxtyping import Float, Int
from typing import Callable
from torch import Tensor
from transformer_lens.hook_points import HookPoint

def mlp_zero_ablation_hook(
    z: Float[Tensor, "seq d_mlp"],
    hook: HookPoint,
) -> None:
    z = torch.zeros_like(z)

In [128]:
def get_ablation_scores(
    model: HookedTransformer,
    tokens: Int[Tensor, "batch seq"],
    hook: Callable = mlp_zero_ablation_hook,
) -> Float[Tensor, "n_layers"]:

    # Initialize an object to store the ablation scores
    ablation_scores = torch.zeros((model.cfg.n_layers,), device=model.cfg.device)

    # Calculating loss without any ablation, to act as a baseline
    model.reset_hooks()
    logits = model(tokens, return_type="logits")
    nex_token_pred = model.to_string(torch.argmax(logits[:, -1, :].squeeze()))
    print(nex_token_pred)

    loss_no_ablation = -get_log_probs(logits, tokens)[:, -(seq_len - 1) :].mean()

    for layer in tqdm(range(model.cfg.n_layers)):
        ablated_logits = model.run_with_hooks(tokens, fwd_hooks=[(utils.get_act_name("z", layer), hook)])
        # Calculate the loss difference (= negative correct logprobs), only on the last `seq_len` tokens
        loss = -get_log_probs(ablated_logits, tokens)[:, -(seq_len - 1) :].mean()
        # Store the result, subtracting the clean loss so that a value of zero means no change in loss
        ablation_scores[layer, head] = loss - loss_no_ablation

    return ablation_scores

get_ablation_scores(gemma2_original, inputs, hook=mlp_zero_ablation_hook)

cdn


NameError: name 'get_log_probs' is not defined

In [126]:
tokenizer.decode(torch.argmax(model(inputs).logits[:,-1,:].squeeze()))

'8'